<h2>C - Network

In [2]:
import pandas as pd
import icecream as ic
import numpy as np
import plotly
import matplotlib as plt
import xml.etree.ElementTree as ET
import pandas as pd
import psycopg2
import psycopg2.extras
import csv

C1: small-molecule roles

In [61]:
#chebi database

hostname = 'localhost'
database = 'Chebi'
username = 'postgres'
pwd = 'wlf1115'
port_id = 5432
conn = None

query = 'Acetazolamide'
with psycopg2.connect(
                    host = hostname,
                    dbname = database,
                    user = username,
                    password = pwd,
                    port = port_id) as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:

        query_script = '''SELECT type, final_id
                        FROM relation
                        WHERE init_id = (
                            SELECT compound_id
                            FROM names
                            WHERE name IN (%s)
                            )'''
        cur.execute(query_script, (query,))
        
        type_id = {}
        type_compound = {}
        for record in cur.fetchall():
            type_id[record[0]] = []
            type_compound[record[0]] = []
            type_id[record[0]] = record[1]


        for k,v in type_id.items():
            # query1 = '''SELECT name
            #             FROM names
            #             WHERE compound_id IN ({}
            #             )'''.format(tuple([x]))
            query_script1 = '''SELECT name
                        FROM names
                        WHERE compound_id IN (%s)
                        '''
            query1 = tuple([v])
            cur.execute(query_script1, query1)
            for record in cur.fetchall():
                type_compound[k].append(record)

print(query)
type_compound    

Acetazolamide


{'has_part': [['Diamox'],
  ['Sodium acetazolamide'],
  ['sodium [(5-acetamido-1,3,4-thiadiazol-2-yl)sulfonyl]azanide'],
  ['N-(5-Sulfamoyl-1,3,4-thiadiazol-2-yl)acetamide monosodium salt']],
 'is_conjugate_base_of': [['[(5-acetamido-1,3,4-thiadiazol-2-yl)sulfonyl]azanide'],
  ['acetazolamide']]}

C2: metabolic pathways

In [ ]:
# https://www.ebi.ac.uk/biomodels/MODEL1109130000#Files #!how to get binary data, how is the graph represented # read paper
# We downloaded the reconstruction of human metabolism (Recon)57 from Pathway Commons58 (http://www.pathwaycommons.org, July 2017) in binary interaction form. 
# Data were represented as an undirected graph where nodes are metabolites and edges denote reactions. 

In [120]:
recon_df = pd.read_csv("../PathwayCommons.8.reconx.BINARY_SIF.hgnc.txt", sep='\t', names=['init', 'process','affected'])
recon_df #how to convert protein name to abbreviation

,init,process,affected
0,A4GALT,catalysis-precedes,B3GALNT1
1,A4GALT,catalysis-precedes,B3GNT3
2,A4GALT,controls-production-of,CHEBI:58223
3,A4GALT,catalysis-precedes,ENTPD4
4,A4GALT,catalysis-precedes,NME1
...,...,...,...
52598,ZADH2,controls-production-of,CHEBI:17790
52599,ZADH2,controls-production-of,CHEBI:18041
52600,ZADH2,controls-production-of,CHEBI:28972
52601,ZADH2,catalysis-precedes,GLO1


C3: signaling pathways

In [ ]:
#  C3 space (and the C4 and C5 spaces) is aimed at any molecule with known protein targets. 
# In this case, we list the biological pathways that may be affected by the interaction of a molecule with its targets. 
# Human pathways were collected from Reactome (https://reactome.org, May 2017) #! use of graphDB

# we chose to use binding activities from B4, since this is an extensive dataset containing mostly literature data with well-accepted activity thresholds. 
# In B4, 24.5% of the compound–protein interactions do not correspond to human proteins. #!need B4 data
# These were mapped to their human orthologs using MetaPhOrs60 (http://orthology.phylomedb.org, May 2017), following the observation that binding activities can be safely transferred between orthologous proteins61,  #!how is it done, it is a database?
# especially if they belong to closely related species, as it is the case for B4 data62. Of all the nonhuman proteins mapped to the human orthologs, 94.4% were mammal proteins #!http://orthology.phylomedb.org/download

In [7]:
reactome_df = pd.read_csv("../UniProt2Reactome_All_Levels.txt", sep='\t', names=['uniprot_id','interaction_code','web','pathway','IEA','organism'])
reactome_df = reactome_df[['uniprot_id','pathway']]

chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
chemble_B1_target_uniprot = chemble_B1[chemble_B1['compound_name']=='Acetazolamide']['uniprot_id']



/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_1245/1950680321.py:4: DtypeWarning: Columns (5,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)


In [18]:
#  biological pathways that may be affected by the interaction of a molecule with its targets. 
# ! still need to make sure all the non human genes are turned into human orthologs
reactome_df[reactome_df['uniprot_id'].isin(chemble_B1_target_uniprot)]


,uniprot_id,pathway
412824,O00255,Signal Transduction
412825,O00255,Signaling by TGF-beta Receptor Complex
412826,O00255,Signaling by Rho GTPases
412827,O00255,RHO GTPase Effectors
412828,O00255,Signaling by WNT
...,...,...
903808,Q9Y6L6,Disorders of transmembrane transporters
903809,Q9Y6L6,Transport of organic anions
903810,Q9Y6L6,Metabolism of steroids
903811,Q9Y6L6,Drug ADME


C4: biological processes

In [6]:
with open('../GOA/25.H_sapiens.goa', 'r') as handle:

    for rec in gafiterator(handle):
        print(rec)
        break

{'DB': 'UniProtKB', 'DB_Object_ID': 'A0A024QZ33', 'DB_Object_Symbol': 'NSRP1', 'Qualifier': ['involved_in'], 'GO_ID': 'GO:0000381', 'DB:Reference': ['GO_REF:0000002'], 'Evidence': 'IEA', 'With': ['InterPro:IPR018612'], 'Aspect': 'P', 'DB_Object_Name': 'Nuclear speckle splicing regulatory protein 1', 'Synonym': ['NSRP1', 'CCDC55', 'hCG_1646942'], 'DB_Object_Type': 'protein', 'Taxon_ID': ['9606'], 'Date': '20220907', 'Assigned_By': 'InterPro', 'Annotation_Extension': '', 'Gene_Product_Form_ID': ''}


In [17]:
# We downloaded the Gene Ontology Annotation database (https://www.ebi.ac.uk/GOA, May 2017)
# read the ‘biological process’ branch of the ontology as a directed acyclic graph (‘is a’ relationships). 
# Proteins were annotated with their Gene Ontology Annotation biological process terms plus parent terms (up to the root of the directed acyclic graph). 
# Similar to C3, we associated molecules with biological process terms by simply checking the annotations of the molecule targets (B4).
from ast import And
from Bio.UniProt.GOA import gafiterator, record_has

for x in chemble_B1_target_uniprot.to_list():
    Interpro_ID = {'With': set([f'InterPro:IPR018612'])} 
    DB_Object_ID = {'DB_Object_ID': set([x])} 

    # DB_Object_name = {'DB_Object_name' : set(['Nuclear speckle splicing regulatory protein 1'])} 

    with open('../GOA/25.H_sapiens.goa', 'r') as handle:

        for rec in gafiterator(handle):
            if record_has(rec, DB_Object_ID):
                for key in ('DB_Object_ID','DB_Object_Name', 'Qualifier', 'GO_ID'):
                    print(rec[key])
                print('='*20)
        break

P00915
Carbonic anhydrase 1
['enables']
GO:0004064
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0004089
P00915
Carbonic anhydrase 1
['enables']
GO:0005515
P00915
Carbonic anhydrase 1
['enables']
GO:0005515
P00915
Carbonic anhydrase 1
['enables']
GO:0008270
P00915
Carbonic anhydrase 1
['enables']
GO:0016829
P00915
Carbonic anhydrase 1
['enables']
GO:0016836
P00915
Carbonic anhydrase 1
['enables']
GO:0016836
P00915
Carbonic anhydrase 1
['enables']
GO:0018820
P00915
Carbonic anhydrase 1
['enables']
GO:0046872
P00915
Carbonic anhydrase 1
['involved_in']
GO:0006730
P00915
Carbonic anhydrase 1
['involved_in']
GO:0006730
P00915
Carbonic anhydrase 1
['is_active_in']
GO:0005737
P00915
Carbonic anhydrase 1
['located_in']
GO:0005737
P00915
Carbonic

C5: interactomes

In [68]:
# We collected five representative PPI networks, 
# namely STRING (score of >700, that is, high confidence)63 (v.10, https://string-db.org) (14,725 proteins (p), 300,686 interactions (i)), #! waiting for data
# InWeb (score of 0.5)64 (http://www.intomics.com/inbio/map, March 2017) (10,100 p, 168,970 i), #!paid service? <- not gonna use
# a portion of Pathway Commons containing interactions from known pathways (Kyoto Encylopedia of Genes and Genomes65, NetPath66, PANTHER67 and WikiPathways68) (9,344 p, 242,962 i), #!need to find out
# an in-house network of physical binary PPIs69 (13,038 p, 64,659 i),  #!need to find in house pipeline

In [113]:
stringDB_uniprot = pd.read_csv('../human_HighConfidence_string_PPI_add_uniprot.csv')
stringDB_uniprot.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_uniprot_id,protein1_protein_name,protein2_uniprot_id,protein2_protein_name
0,9606.ENSP00000000233,9606.ENSP00000324287,0,0,0,49,147,0,736,767,P84085,ADP-ribosylation factor 5,Q15057,"Arf-GAP with coiled-coil, ANK repeat and PH do..."
1,9606.ENSP00000000233,9606.ENSP00000387286,0,0,0,79,248,600,141,730,P84085,ADP-ribosylation factor 5,P62820,Ras-related protein Rab-1A (EC 3.6.5.2) (YPT1-...
2,9606.ENSP00000000233,9606.ENSP00000262812,0,0,0,190,163,600,173,745,P84085,ADP-ribosylation factor 5,O14579,Coatomer subunit epsilon (Epsilon-coat protein...
3,9606.ENSP00000000233,9606.ENSP00000158762,0,0,0,0,147,0,736,765,P84085,ADP-ribosylation factor 5,Q15027,"Arf-GAP with coiled-coil, ANK repeat and PH do..."
4,9606.ENSP00000000233,9606.ENSP00000449270,0,0,0,162,210,600,194,757,P84085,ADP-ribosylation factor 5,P61923,Coatomer subunit zeta-1 (Zeta-1-coat protein) ...


In [8]:
stringDB_df = pd.read_csv('../human_HighConfidence_string_PPI.csv', sep=' ')

In [ ]:
#create input for hotnet -stringDB
bla = np.append(stringDB_df['protein1'].unique(), stringDB_df['protein2'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
bla = set(bla.tolist())

protein_encoding = {}

for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in stringDB_df['protein1']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in stringDB_df['protein2']:
    protein2_encoding.append(protein_encoding[x])
    
stringDB_df['protein1_encoding'] = protein1_encoding
stringDB_df['protein2_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('stringDB_index.txt', sep = " ", index = False, index_label = False, header = False)
stringDB_df[['protein1_encoding','protein2_encoding']].to_csv('stringDB_edge.txt', sep = " ", index = False, index_label = False, header = False)

In [18]:
stringDB_uniprot['protein2_uniprot_id']

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_uniprot_id,protein1_protein_name,protein2_uniprot_id,protein2_protein_name,protein1_encoding,protein2_encoding
0,9606.ENSP00000000233,9606.ENSP00000324287,0,0,0,49,147,0,736,767,P84085,ADP-ribosylation factor 5,Q15057,"Arf-GAP with coiled-coil, ANK repeat and PH do...",9087,2029
1,9606.ENSP00000000233,9606.ENSP00000387286,0,0,0,79,248,600,141,730,P84085,ADP-ribosylation factor 5,P62820,Ras-related protein Rab-1A (EC 3.6.5.2) (YPT1-...,9087,11094
2,9606.ENSP00000000233,9606.ENSP00000262812,0,0,0,190,163,600,173,745,P84085,ADP-ribosylation factor 5,O14579,Coatomer subunit epsilon (Epsilon-coat protein...,9087,2130
3,9606.ENSP00000000233,9606.ENSP00000158762,0,0,0,0,147,0,736,765,P84085,ADP-ribosylation factor 5,Q15027,"Arf-GAP with coiled-coil, ANK repeat and PH do...",9087,14432
4,9606.ENSP00000000233,9606.ENSP00000449270,0,0,0,162,210,600,194,757,P84085,ADP-ribosylation factor 5,P61923,Coatomer subunit zeta-1 (Zeta-1-coat protein) ...,9087,15393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504021,9606.ENSP00000485663,9606.ENSP00000248342,0,0,0,699,996,900,957,999,Q9Y262,Eukaryotic translation initiation factor 3 sub...,Q9UBQ5,Eukaryotic translation initiation factor 3 sub...,6259,8319
504022,9606.ENSP00000485663,9606.ENSP00000416255,0,0,0,167,808,0,473,908,Q9Y262,Eukaryotic translation initiation factor 3 sub...,Q04637,Eukaryotic translation initiation factor 4 gam...,6259,1994
504023,9606.ENSP00000485663,9606.ENSP00000220849,0,0,0,907,993,900,896,999,Q9Y262,Eukaryotic translation initiation factor 3 sub...,P60228,Eukaryotic translation initiation factor 3 sub...,6259,213
504024,9606.ENSP00000485663,9606.ENSP00000346088,0,0,0,711,0,0,131,738,Q9Y262,Eukaryotic translation initiation factor 3 sub...,P35268,60S ribosomal protein L22 (EBER-associated pro...,6259,6899


In [19]:
#create input for hotnet -stringDB
bla = np.append(stringDB_uniprot['protein1_uniprot_id'].unique(), stringDB_uniprot['protein2_uniprot_id'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
bla = set(bla.tolist())

protein_encoding = {}

for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in stringDB_uniprot['protein1_uniprot_id']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in stringDB_uniprot['protein2_uniprot_id']:
    protein2_encoding.append(protein_encoding[x])
    
stringDB_uniprot['protein1_encoding'] = protein1_encoding
stringDB_uniprot['protein2_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('stringDB_index1.txt', sep = " ", index = False, index_label = False, header = False)
stringDB_uniprot[['protein1_encoding','protein2_encoding']].to_csv('stringDB_edge1.txt', sep = " ", index = False, index_label = False, header = False)

In [16]:
stringDB_uniprot.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_uniprot_id,protein1_protein_name,protein2_uniprot_id,protein2_protein_name,protein1_encoding,protein2_encoding
0,9606.ENSP00000000233,9606.ENSP00000324287,0,0,0,49,147,0,736,767,P84085,ADP-ribosylation factor 5,Q15057,"Arf-GAP with coiled-coil, ANK repeat and PH do...",9087,2029
1,9606.ENSP00000000233,9606.ENSP00000387286,0,0,0,79,248,600,141,730,P84085,ADP-ribosylation factor 5,P62820,Ras-related protein Rab-1A (EC 3.6.5.2) (YPT1-...,9087,11094
2,9606.ENSP00000000233,9606.ENSP00000262812,0,0,0,190,163,600,173,745,P84085,ADP-ribosylation factor 5,O14579,Coatomer subunit epsilon (Epsilon-coat protein...,9087,2130
3,9606.ENSP00000000233,9606.ENSP00000158762,0,0,0,0,147,0,736,765,P84085,ADP-ribosylation factor 5,Q15027,"Arf-GAP with coiled-coil, ANK repeat and PH do...",9087,14432
4,9606.ENSP00000000233,9606.ENSP00000449270,0,0,0,162,210,600,194,757,P84085,ADP-ribosylation factor 5,P61923,Coatomer subunit zeta-1 (Zeta-1-coat protein) ...,9087,15393


In [120]:
stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id']=='P00915']

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_uniprot_id,protein1_protein_name,protein2_uniprot_id,protein2_protein_name
451292,9606.ENSP00000430656,9606.ENSP00000427690,0,0,0,722,0,0,96,737,P00915,Carbonic anhydrase 1 (EC 4.2.1.1) (Carbonate d...,P06028,Glycophorin-B (PAS-3) (SS-active sialoglycopro...
451293,9606.ENSP00000430656,9606.ENSP00000376134,0,0,0,0,0,900,0,900,P00915,Carbonic anhydrase 1 (EC 4.2.1.1) (Carbonate d...,Q14765,Signal transducer and activator of transcripti...
451294,9606.ENSP00000430656,9606.ENSP00000307199,0,0,0,774,0,0,323,840,P00915,Carbonic anhydrase 1 (EC 4.2.1.1) (Carbonate d...,Q9NZD4,Alpha-hemoglobin-stabilizing protein (Erythroi...
451295,9606.ENSP00000430656,9606.ENSP00000300215,0,0,0,720,0,0,310,798,P00915,Carbonic anhydrase 1 (EC 4.2.1.1) (Carbonate d...,P16452,Protein 4.2 (P4.2) (Erythrocyte membrane prote...
451296,9606.ENSP00000430656,9606.ENSP00000262418,0,0,0,726,56,0,219,780,P00915,Carbonic anhydrase 1 (EC 4.2.1.1) (Carbonate d...,P02730,Band 3 anion transport protein (Anion exchange...
451297,9606.ENSP00000430656,9606.ENSP00000332369,0,0,0,722,0,0,232,777,P00915,Carbonic anhydrase 1 (EC 4.2.1.1) (Carbonate d...,P22557,"5-aminolevulinate synthase, erythroid-specific..."
451298,9606.ENSP00000430656,9606.ENSP00000369654,0,0,0,645,0,0,390,774,P00915,Carbonic anhydrase 1 (EC 4.2.1.1) (Carbonate d...,P02042,Hemoglobin subunit delta (Delta-globin) (Hemog...
451299,9606.ENSP00000430656,9606.ENSP00000216862,0,0,0,0,0,0,870,870,P00915,Carbonic anhydrase 1 (EC 4.2.1.1) (Carbonate d...,Q07973,"1,25-dihydroxyvitamin D(3) 24-hydroxylase, mit..."


In [61]:
stringDB_df

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_encoding
0,9606.ENSP00000000233,9606.ENSP00000324287,0,0,0,49,147,0,736,767,1
1,9606.ENSP00000000233,9606.ENSP00000387286,0,0,0,79,248,600,141,730,2
2,9606.ENSP00000000233,9606.ENSP00000262812,0,0,0,190,163,600,173,745,3
3,9606.ENSP00000000233,9606.ENSP00000158762,0,0,0,0,147,0,736,765,4
4,9606.ENSP00000000233,9606.ENSP00000449270,0,0,0,162,210,600,194,757,5
...,...,...,...,...,...,...,...,...,...,...,...
504021,9606.ENSP00000485663,9606.ENSP00000248342,0,0,0,699,996,900,957,999,504022
504022,9606.ENSP00000485663,9606.ENSP00000416255,0,0,0,167,808,0,473,908,504023
504023,9606.ENSP00000485663,9606.ENSP00000220849,0,0,0,907,993,900,896,999,504024
504024,9606.ENSP00000485663,9606.ENSP00000346088,0,0,0,711,0,0,131,738,504025


In [70]:
stringDB_df = stringDB_df[stringDB_df['combined_score']>700]

In [ ]:
def Intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

# Driver Code
print(Intersection(stringDB_df['protein1'].unique(), stringDB_df['protein2'].unique()))

In [20]:
# an in-house network of physical binary PPIs69 (13,038 p, 64,659 i),  #!need to find in house pipeline
import csv
# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open("../interactions_interactome_3d.dat").readlines()]
interaction3D_df = pd.DataFrame(datContent, columns = datContent[0])
interaction3D_df = interaction3D_df[1:] 
interaction3D_df #! how did you get binary PPI interactome info from this?  https://interactome3d.irbbarcelona.org/help.php#interaction_page
# https://interactome3d.irbbarcelona.org/help.php#interaction_page

,PROT1,PROT2,RANK_MAJOR,RANK_MINOR,TYPE,PDB_ID,BIO_UNIT,CHAIN1,MODEL1,SEQ_IDENT1,...,SEQ_END1,DOMAIN1,CHAIN2,MODEL2,SEQ_IDENT2,COVERAGE2,SEQ_BEGIN2,SEQ_END2,DOMAIN2,FILENAME
1,A0A024RAV5,P02647,1,0,Structure,2mse,1,B,0,100.0,...,185,-,A,0,100.0,74.2,68,265,-,A0A024RAV5-P02647-EXP-2mse.pdb1-B-0-A-0.pdb
2,A0A024RAV5,P02647,2,0,Structure,6pts,1,B,0,100.0,...,185,-,A,0,100.0,74.2,68,265,-,A0A024RAV5-P02647-EXP-6pts.pdb1-B-0-A-0.pdb
3,A0A024RAV5,P10398,1,0,Structure,2mse,1,B,0,100.0,...,185,-,D,0,100.0,12.0,19,91,-,A0A024RAV5-P10398-EXP-2mse.pdb1-B-0-D-0.pdb
4,A0A075B5G3,Q8NBP7,1,0,Structure,4ov6,1,F,0,100.0,...,99,-,B,0,95.6,42.5,153,446,-,A0A075B5G3-Q8NBP7-EXP-4ov6.pdb1-F-0-B-0.pdb
5,A0A075B5G3,Q8NBP7,2,0,Structure,4ov6,2,G,0,100.0,...,96,-,E,0,92.2,42.3,153,445,-,A0A075B5G3-Q8NBP7-EXP-4ov6.pdb2-G-0-E-0.pdb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124919,V9HW83,V9HW83,54,0,Structure,4wb9,1,A,2,100.0,...,501,-,A,3,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-2-A-3.pdb
124920,V9HW83,V9HW83,55,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,1,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-1.pdb
124921,V9HW83,V9HW83,56,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,2,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-2.pdb
124922,V9HW83,V9HW83,57,0,Structure,4wb9,1,A,0,100.0,...,501,-,A,3,100.0,98.4,9,501,-,V9HW83-V9HW83-EXP-4wb9.pdb1-A-0-A-3.pdb


In [145]:
bla = np.append(interaction3D_df['PROT1'].unique(), interaction3D_df['PROT2'].unique()) #this needs to be done as there are unique prot in prot2 that don't exist in prot1
bla = set(bla.tolist())

protein_encoding = {}
for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in interaction3D_df['PROT1']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in interaction3D_df['PROT2']:
    protein2_encoding.append(protein_encoding[x])
    
interaction3D_df['PROT1_encoding'] = protein1_encoding
interaction3D_df['PROT2_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('interaction3D_index.txt', sep = " ", index = False, index_label = False, header = False)
interaction3D_df[['PROT1_encoding','PROT2_encoding']].to_csv('interaction3D_edge.txt', sep = " ", index = False, index_label = False, header = False)

In [121]:
import h5py
with h5py.File('/Users/linfengwang/Github/linfeng-wang-CCdatabase/hotnet_output/stringDB_mat_ppr_0.6.h5', "r") as hdf:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % hdf.keys())
    PPR = hdf.get('PPR')
    PPR_dataset = np.array(PPR)
    
    edges = hdf.get('edges') #sample name, given in pairs
    edges_dataset = np.array(edges)

    nodes = hdf.get('nodes') #individual sample names
    nodes_dataset = np.array(nodes) 

    # network_name = hdf.get('network_name') #name of the network
    # network_name_dataset = np.array(network_name)
    # beta = hdf.get('beta') #version beta
    # beta_dataset = np.array(beta)

    # # get first object name/key; may or may NOT be a group
    # a_group_key = list(f.keys())[0]

    # # get the object type for a_group_key: usually group or dataset
    # print(type(f[a_group_key])) 

    # # If a_group_key is a group name, 
    # # this gets the object names in the group and returns as a list
    # data = list(f[a_group_key])

    # # If a_group_key is a dataset name, 
    # # this gets the dataset values and returns as a list
    # data = list(f[a_group_key])
    # # preferred methods to get dataset values:
    # ds_obj = f[a_group_key]      # returns as a h5py dataset object
    # ds_arr = f[a_group_key][()]  # returns as a numpy array

Keys: <KeysViewHDF5 ['PPR', 'beta', 'edges', 'network_name', 'nodes']>


In [131]:


for x in edges_dataset:
    # print(x[0].decode('ascii'))
    if x[0].decode('ascii') == '9606.ENSP00000430656':
        print(x)


[b'9606.ENSP00000430656' b'9606.ENSP00000300215']
[b'9606.ENSP00000430656' b'9606.ENSP00000216862']
[b'9606.ENSP00000430656' b'9606.ENSP00000427690']
[b'9606.ENSP00000430656' b'9606.ENSP00000376134']
[b'9606.ENSP00000430656' b'9606.ENSP00000262418']


In [124]:
edges_dataset

array([[b'9606.ENSP00000460380', b'9606.ENSP00000219476'],
       [b'9606.ENSP00000460380', b'9606.ENSP00000346139'],
       [b'9606.ENSP00000460380', b'9606.ENSP00000353549'],
       ...,
       [b'9606.ENSP00000263578', b'9606.ENSP00000268668'],
       [b'9606.ENSP00000300738', b'9606.ENSP00000346921'],
       [b'9606.ENSP00000355090', b'9606.ENSP00000257934']], dtype='|S20')

In [128]:
for target in target_list:
    for x in edges_dataset:
        # print(x[0].decode('ascii'))
        if x[0].decode('ascii') == target:
            print(x)

<Closed HDF5 dataset>

In [164]:
def influence_node(input_array, r_min=0, r_max=10): #maximum of the range of your measurement
    input_max = np.amax(input_array) #maximum of the range of your desired target scaling
    input_min = np.amin(input_array) #minimum of the range of your desired target scaling
    output = np.ndarray.round((input_array - input_min) / (input_max - input_min) * (r_max - r_min) + r_min)
    return output

In [100]:
prr = influence_node(PPR_dataset)

opening interactome3D h5

In [48]:
import h5py
with h5py.File('/Users/linfengwang/Github/linfeng-wang-CCdatabase/hotnet_output/interaction3D_mat_ppr_0.6.h5', "r") as hdf:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % hdf.keys())
    PPR = hdf.get('PPR')
    PPR_dataset = np.array(PPR)
    
    edges = hdf.get('edges') #sample name, given in pairs
    edges_dataset = np.array(edges)

    nodes = hdf.get('nodes') #individual sample names
    nodes_dataset = np.array(nodes) 


Keys: <KeysViewHDF5 ['PPR', 'beta', 'edges', 'network_name', 'nodes']>


In [132]:
drugbank_df = pd.read_csv("DB/drugbank_extracted.csv")
target_list = drugbank_df[drugbank_df["dg_name"]=="Acetazolamide"]['target_uniprot'].tolist()

In [139]:
target_list = stringDB_uniprot[stringDB_uniprot['protein1_uniprot_id'].isin(target_list)]['protein1'].unique().tolist()

In [ ]:
target_edge

In [167]:
target_edge

[array([b'9606.ENSP00000178638', b'9606.ENSP00000216862'], dtype='|S20'),
 array([b'9606.ENSP00000178638', b'9606.ENSP00000300900'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000262418'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000295256'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000250559'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000216862'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000382004'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000265686'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000261769'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000393557'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000373620'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000378920'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000304669'], dtype='|S20'),
 array([b'9606.ENSP00000285381', b'960

In [171]:
dict_= {}
dict_['a'] = [1,2]

In [181]:
# target_edge = []
# for target in target_list:
#     for x in edges_dataset:
#         # print(x[0].decode('ascii'))
#         if x[0].decode('ascii') == target:
#             target_edge.append(x)
#         elif x[1].decode('ascii') == target:
#             target_edge.append(x)
# is is only protein1 influencing protein2 right? 

In [ ]:
#convert the edge name to index using dictionary
target_edge_coord = {}
for x in target_edge:
    target_edge_coord[x] = ['nan', 'nan']
    for i, y in enumerate(nodes_dataset):
        if x[0] == y:
            target_edge_coord[x]=i
        if x[1] == y:
            target_edge_coord[x]=i

In [193]:
#get the edges
target_edge = []
for target in target_list:
    for x in edges_dataset:
        # print(x[0].decode('ascii'))
        if x[0].decode('ascii') == target:
            target_edge.append(x)
            
# #convert the edge name to index
target_edge_coord = []
for x in target_edge:
    coord = ['nan', 'nan']
    for i, y in enumerate(nodes_dataset):
        if x[0] == y:
            coord[0]=i
        if x[1] == y:
            coord[1]=i
    target_edge_coord.append(coord)
    
# #convert the edge name to index using dictionary
# target_edge_coord = {}
# for x in target_edge:
#     target_edge_coord[x.tolist()] = ['nan', 'nan']
#     for i, y in enumerate(nodes_dataset):
#         if x[0] == y:
#             target_edge_coord[x]=i
#         if x[1] == y:
#             target_edge_coord[x]=i
    
#interaction values extraction using coordinates

interaction_values = []
for x in target_edge_coord:
    interaction_values.append(PPR_dataset[x[0],x[1]])

#scale interaction values to between 0 and 10
influence_values = influence_node(interaction_values)


# Then, for each target of a certain compound, we kept proteins with a non-0 influence score (the target itself was given a score of ten  and, when one protein was influenced by more than one target, the maximum score was kept). 
influenced = {}
for x, y in zip(interaction_values, target_edge):
    if x == 0:
        continue
    if y[1] not in influenced.keys():
        influenced[y[1]] = x
    else:
        if influenced[y[1]] < x:
            influenced[y[1]] = x
        else:
            continue
    if y[0] not in influenced.keys():
        influenced[y[0]] = 10
    
    

In [203]:
influenced.keys()

TypeError: 'dict_keys' object is not subscriptable

In [207]:
chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
chemble_B4B5 = pd.read_csv("../chembl31forlinfeng/B4B5_data.csv", sep=',', header=0)
chemble_B4B5_ic50 = chemble_B4B5[chemble_B4B5['standard_type']=='IC50'] #B4

ion_channels = ['Ligand-gated ion channel','Voltage-gated ion channel', 'Other ion channel']
g_protein = ['Family A G protein-coupled receptor', 'Family C G protein-coupled receptor',
            'Family B G protein-coupled receptor','Frizzled family G protein-coupled receptor',
            'Taste family G protein-coupled receptor']

not_other = ['Ligand-gated ion channel','Voltage-gated ion channel', 'Other ion channel'
            'Family A G protein-coupled receptor', 'Family C G protein-coupled receptor',
            'Family B G protein-coupled receptor','Frizzled family G protein-coupled receptor',
            'Taste family G protein-coupled receptor', 'Kinase','Nuclear receptor']
# kinases ≤30 nM, 
kinase_df = chemble_B1[chemble_B1['l2']=="Kinase"]
kinase_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(kinase_df['uniprot_id'])]
kinase_ic_filter = chemble_B4B5_ic50[chemble_B4B5_ic50['standard_value']<=30]
# G protein-coupled receptors ≤100 nM, 
g_protein_df = chemble_B1[chemble_B1['l2'].isin(g_protein)]
g_protein_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(g_protein_df['uniprot_id'])]
g_protein_ic_filter = chemble_B4B5_ic50[chemble_B4B5_ic50['standard_value']<=100]
# nuclear receptors ≤100 nM,  
nuclear_receptor_df = chemble_B1[chemble_B1['l2']=='Nuclear receptor']
nuclear_receptor_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(nuclear_receptor_df['uniprot_id'])]
nuclear_receptor_ic_filter = chemble_B4B5_ic50[chemble_B4B5_ic50['standard_value']<=100]
# ion channels ≤10 μM
ion_channel_df = chemble_B1[chemble_B1['l2'].isin(ion_channels)]
ion_channel_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(ion_channel_df['uniprot_id'])]
ion_channel_ic_filter = chemble_B4B5_ic50[chemble_B4B5_ic50['standard_value']<=1000]
# others ≤1 μM. 

others_df = chemble_B1[~chemble_B1['l2'].isin(not_other)]
other_ic = chemble_B4B5_ic50[chemble_B4B5_ic50['uniprot_id'].isin(others_df['uniprot_id'])] #!数据重复是怎么回事
other_ic_filter = other_ic[other_ic['standard_value']<=1000]
# We also kept activities one order of magnitude lower than the class-specific cutoff (to a maximum of 10 μM), 
# and gave these annotations half the weight in downstream analyses (that is, log10 scaling)

in_threshold = pd.concat([kinase_ic_filter, g_protein_ic_filter,nuclear_receptor_ic_filter,ion_channel_ic_filter,other_ic_filter])


/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_11579/625214163.py:1: DtypeWarning: Columns (5,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B1 = pd.read_csv("../chembl31forlinfeng/B1_data.csv", sep=',', header=0)
/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_11579/625214163.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  chemble_B4B5 = pd.read_csv("../chembl31forlinfeng/B4B5_data.csv", sep=',', header=0)
/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_11579/625214163.py:38: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  in_threshold = pd.concat(kinase_ic_filter, g_protein_ic_filter,nuclear_receptor_ic_filter,ion_channel_ic_filter,other_ic_filter)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
# Finally, these scores were multiplied by the weight of the compound-target annotation (see B4). 
#is everthing that with in class specific cut of multiplied by 1 and everything outside the class specific cutoff multiplied by log10
for k, v in influenced.items():
    uniprot = stringDB_uniprot[stringDB_uniprot['protein1']==k.decode('ascii')]['protein1_uniprot_id'].unique()[0]
    if uniprot in in_threshold['uniprot_id'].unique():
        continue
    else:
        influenced[k] = np.log10(v)

In [212]:
influenced

{b'9606.ENSP00000216862': 0.015335382,
 b'9606.ENSP00000178638': 10,
 b'9606.ENSP00000300900': 0.0425477,
 b'9606.ENSP00000262418': -1.8118263,
 b'9606.ENSP00000285379': 10,
 b'9606.ENSP00000295256': 0.0023837313,
 b'9606.ENSP00000250559': -2.6582344,
 b'9606.ENSP00000382004': -2.7136254,
 b'9606.ENSP00000265686': -2.3814678,
 b'9606.ENSP00000261769': -2.916688,
 b'9606.ENSP00000393557': -1.6337378,
 b'9606.ENSP00000373620': 0.0017948113,
 b'9606.ENSP00000378920': -1.635768,
 b'9606.ENSP00000304669': -2.6967418,
 b'9606.ENSP00000366641': -1.5092093,
 b'9606.ENSP00000285381': 10,
 b'9606.ENSP00000358107': 0.1276431,
 b'9606.ENSP00000367608': 0.017717116,
 b'9606.ENSP00000405899': -1.5747824,
 b'9606.ENSP00000311165': 1.0,
 b'9606.ENSP00000264938': 0.009327489,
 b'9606.ENSP00000318355': -1.4979736,
 b'9606.ENSP00000220764': -1.9046055,
 b'9606.ENSP00000345659': 10,
 b'9606.ENSP00000300215': -1.707651,
 b'9606.ENSP00000430656': 10,
 b'9606.ENSP00000427690': -1.6068602,
 b'9606.ENSP0000037

In [195]:
stringDB_uniprot.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_uniprot_id,protein1_protein_name,protein2_uniprot_id,protein2_protein_name
0,9606.ENSP00000000233,9606.ENSP00000324287,0,0,0,49,147,0,736,767,P84085,ADP-ribosylation factor 5,Q15057,"Arf-GAP with coiled-coil, ANK repeat and PH do..."
1,9606.ENSP00000000233,9606.ENSP00000387286,0,0,0,79,248,600,141,730,P84085,ADP-ribosylation factor 5,P62820,Ras-related protein Rab-1A (EC 3.6.5.2) (YPT1-...
2,9606.ENSP00000000233,9606.ENSP00000262812,0,0,0,190,163,600,173,745,P84085,ADP-ribosylation factor 5,O14579,Coatomer subunit epsilon (Epsilon-coat protein...
3,9606.ENSP00000000233,9606.ENSP00000158762,0,0,0,0,147,0,736,765,P84085,ADP-ribosylation factor 5,Q15027,"Arf-GAP with coiled-coil, ANK repeat and PH do..."
4,9606.ENSP00000000233,9606.ENSP00000449270,0,0,0,162,210,600,194,757,P84085,ADP-ribosylation factor 5,P61923,Coatomer subunit zeta-1 (Zeta-1-coat protein) ...


In [194]:
influenced

{b'9606.ENSP00000216862': 0.015335382,
 b'9606.ENSP00000178638': 10,
 b'9606.ENSP00000300900': 0.0425477,
 b'9606.ENSP00000262418': 0.015423169,
 b'9606.ENSP00000285379': 10,
 b'9606.ENSP00000295256': 0.0023837313,
 b'9606.ENSP00000250559': 0.0021966745,
 b'9606.ENSP00000382004': 0.0019336351,
 b'9606.ENSP00000265686': 0.004154629,
 b'9606.ENSP00000261769': 0.0012114684,
 b'9606.ENSP00000393557': 0.023241397,
 b'9606.ENSP00000373620': 0.0017948113,
 b'9606.ENSP00000378920': 0.023133002,
 b'9606.ENSP00000304669': 0.0020102877,
 b'9606.ENSP00000366641': 0.03095927,
 b'9606.ENSP00000285381': 10,
 b'9606.ENSP00000358107': 0.1276431,
 b'9606.ENSP00000367608': 0.017717116,
 b'9606.ENSP00000405899': 0.026620587,
 b'9606.ENSP00000311165': 10,
 b'9606.ENSP00000264938': 0.009327489,
 b'9606.ENSP00000318355': 0.031770673,
 b'9606.ENSP00000220764': 0.012456457,
 b'9606.ENSP00000345659': 10,
 b'9606.ENSP00000300215': 0.019604191,
 b'9606.ENSP00000430656': 10,
 b'9606.ENSP00000427690': 0.024725204,


In [191]:
target_edge

[array([b'9606.ENSP00000178638', b'9606.ENSP00000216862'], dtype='|S20'),
 array([b'9606.ENSP00000178638', b'9606.ENSP00000300900'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000262418'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000295256'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000250559'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000216862'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000382004'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000265686'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000261769'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000393557'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000373620'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000378920'], dtype='|S20'),
 array([b'9606.ENSP00000285379', b'9606.ENSP00000304669'], dtype='|S20'),
 array([b'9606.ENSP00000285381', b'960

In [190]:
interaction_values

[0.0113869095,
 0.0425477,
 0.013658639,
 0.0023837313,
 0.0021966745,
 0.010492272,
 0.0019336351,
 0.004154629,
 0.0012114684,
 0.023241397,
 0.0017948113,
 0.023133002,
 0.0020102877,
 0.03095927,
 0.010514199,
 0.1276431,
 0.015335382,
 0.017717116,
 0.026620587,
 0.009327489,
 0.031770673,
 0.012456457,
 0.010656548,
 0.0113869095,
 0.019604191,
 0.010454002,
 0.024725204,
 0.002763526,
 0.015423169]

In [143]:
for target in target_list:
    for i, x in enumerate(nodes_dataset):
        # print(x[0].decode('ascii'))
        if x.decode('ascii') == target:
            print(i,x)

170 b'9606.ENSP00000178638'
3770 b'9606.ENSP00000285379'
3771 b'9606.ENSP00000285381'
4707 b'9606.ENSP00000300900'
5512 b'9606.ENSP00000311165'
7893 b'9606.ENSP00000345659'
9419 b'9606.ENSP00000358107'
14711 b'9606.ENSP00000430656'


In [97]:
# scaling to between 0 and 10 - https://stats.stackexchange.com/questions/281162/scale-a-number-between-a-range

r_min = 90 #minimum of the range of your measurement
r_max = 900 #maximum of the range of your measurement
input_max = 10 #maximum of the range of your desired target scaling
input_min = 0 #minimum of the range of your desired target scaling
input_value = 800
int((input_value - r_min) / (r_max - r_min) * (input_max - input_min) + input_min)

8

In [30]:
# a portion of Pathway Commons containing interactions from known pathways (Kyoto Encylopedia of Genes and Genomes65, NetPath66, PANTHER67 and WikiPathways68) (9,344 p, 242,962 i), #!need to find out
panther = pd.read_csv('../C5_PathwayCommons13.panther.hgnc.txt', sep='\t')
netpath = pd.read_csv('../C5_PathwayCommons13.netpath.hgnc.txt', sep = '\t')

netpath = netpath[netpath['INTERACTION_DATA_SOURCE']=='NetPath']
panther = panther[panther['INTERACTION_DATA_SOURCE']=='PANTHER']
# pathway_common = pd.concat([panther, netpath])

pathway_common = pd.read_csv('../C5_PathwayCommons13.All.hgnc.txt', sep='\t')


In [34]:
panther.head()

,PARTICIPANT_A,INTERACTION_TYPE,PARTICIPANT_B,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID,PATHWAY_NAMES,MEDIATOR_IDS
0,ABAT,catalysis-precedes,ALDH5A1,PANTHER,NaN,Gamma-aminobutyric acid synthesis,http://pathwaycommons.org/pc13/Catalysis_c5d26...
1,ABAT,catalysis-precedes,ALDH6A1,PANTHER,NaN,Pyrimidine Metabolism,http://pathwaycommons.org/pc13/Catalysis_80d0d...
2,ABAT,catalysis-precedes,CSAD,PANTHER,NaN,Gamma-aminobutyric acid synthesis,http://pathwaycommons.org/pc13/Catalysis_c5d26...
3,ABAT,catalysis-precedes,GAD1,PANTHER,NaN,Gamma-aminobutyric acid synthesis,http://pathwaycommons.org/pc13/Catalysis_c5d26...
4,ABAT,catalysis-precedes,GAD2,PANTHER,NaN,Gamma-aminobutyric acid synthesis,http://pathwaycommons.org/pc13/Catalysis_c5d26...


In [14]:
pathway_common = pathway_common[pathway_common['INTERACTION_DATA_SOURCE'].isin(['NetPath','PANTHER'])]

In [29]:
pathway_common.head()

,PARTICIPANT_A,INTERACTION_TYPE,PARTICIPANT_B,INTERACTION_DATA_SOURCE,INTERACTION_PUBMED_ID,PATHWAY_NAMES,MEDIATOR_IDS
2969,ABAT,catalysis-precedes,CSAD,PANTHER,NaN,Gamma-aminobutyric acid synthesis,http://pathwaycommons.org/pc13/Catalysis_c5d26...
12818,ACLY,catalysis-precedes,CS,PANTHER,NaN,Pyruvate metabolism,http://pathwaycommons.org/pc13/BiochemicalReac...
19109,ACTA1,in-complex-with,AFDN,PANTHER,NaN,Alzheimer disease-presenilin pathway,http://pathwaycommons.org/pc13/Complex_c863d39...
19117,ACTA1,in-complex-with,AP2A1,PANTHER,NaN,Huntington disease,http://pathwaycommons.org/pc13/Complex_a545101...
19118,ACTA1,in-complex-with,AP2A2,PANTHER,NaN,Huntington disease,http://pathwaycommons.org/pc13/Complex_a545101...


In [49]:
bla = np.append(pathway_common['PARTICIPANT_A'].unique(), pathway_common['PARTICIPANT_B'].unique()) #this needs to be done as there are unique prot in PARTICIPANT_B that don't exist in PARTICIPANT_A
bla = set(bla.tolist())

protein_encoding = {}
for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in pathway_common['PARTICIPANT_A']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in pathway_common['PARTICIPANT_B']:
    protein2_encoding.append(protein_encoding[x])
    
pathway_common['PARTICIPANT_A_encoding'] = protein1_encoding
pathway_common['PARTICIPANT_B_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('pathway_common_index.txt', sep = " ", index = False, index_label = False, header = False)
pathway_common[['PARTICIPANT_A_encoding','PARTICIPANT_B_encoding']].to_csv('pathway_common_edge.txt', sep = " ", index = False, index_label = False, header = False)

In [44]:
bla = np.append(panther['PARTICIPANT_A'].unique(), panther['PARTICIPANT_B'].unique()) #this needs to be done as there are unique prot in PARTICIPANT_B that don't exist in PARTICIPANT_A
bla = set(bla.tolist())

protein_encoding = {}
for i, x in enumerate(bla):
    protein_encoding[x] = i+1
protein1_encoding = []
for x in panther['PARTICIPANT_A']:
    protein1_encoding.append(protein_encoding[x])
    
protein2_encoding = []
for x in panther['PARTICIPANT_B']:
    protein2_encoding.append(protein_encoding[x])
    
panther['PARTICIPANT_A_encoding'] = protein1_encoding
panther['PARTICIPANT_B_encoding'] = protein2_encoding

protein_encoding = pd.DataFrame({'Names':list(protein_encoding.keys()), 'Encodings':list(protein_encoding.values())})

#export index file
protein_encoding[['Encodings','Names']].to_csv('panther_index.txt', sep = " ", index = False, index_label = False, header = False)
panther[['PARTICIPANT_A_encoding','PARTICIPANT_B_encoding']].to_csv('panther_edge.txt', sep = " ", index = False, index_label = False, header = False)

/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_3221/1536042582.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panther['PARTICIPANT_A_encoding'] = protein1_encoding
/var/folders/78/zn54_c9d2jb_8j8xx5s59dqw0000gn/T/ipykernel_3221/1536042582.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panther['PARTICIPANT_B_encoding'] = protein2_encoding


In [6]:
# a network of metabolic genes based on Recon (v.2, http://vmh.uni.lu) (1,628 p, 246,937 i). #! recon https://www.vmh.life/#home matlab file?
import scipy.io
Recon3DModel_mat =  ('../Recon3D_301/Recon3DModel_301.mat')
Recon3D_mat = scipy.io.loadmat('../Recon3D_301/Recon3D_301.mat')

# To build this last network, we linked two metabolic proteins (enzymes or transporters) when the product metabolite of the first was the substrate of the second, 
# or when both were needed to perform a certain reaction, suggesting that they are part of the same protein complex. 
# Edges between proteins were weighted inversely proportional to the number of reactions involving their shared metabolites, 
# so that ‘currency’ metabolites such as ATP and water had marginal impact on the network connectivity. To control for indirect associations, 
# we deconvoluted the network using edge weights and setting a network deconvolution score cutoff of 0.9

In [4]:
Recon3D_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Recon3D'])

In [7]:
with open('DB/recon_3d.txt', 'w') as f:
    writer = csv.writer(f)
    for x in range(1,len(Recon3D_mat['Recon3D'][0][0])):
        writer.writerow(Recon3D_mat['Recon3D'][0][0][x].tolist())


In [ ]:
Recon3D_mat['Recon3D'][0][0][6].tolist()

In [94]:
Recon3D_mat['Recon3D'][0][0][4]

array([[array(['10FTHF5GLUtl'], dtype='<U12')],
       [array(['10FTHF5GLUtm'], dtype='<U12')],
       [array(['10FTHF6GLUtl'], dtype='<U12')],
       ...,
       [array(['NADH2_u10mi'], dtype='<U11')],
       [array(['CYOOm3i'], dtype='<U7')],
       [array(['CYOOm2i'], dtype='<U7')]], dtype=object)

In [93]:
len(Recon3D_mat['Recon3D'][0][0][1])

8399

In [ ]:
Recon3DModel_mat

In [65]:
#network processing


Hot net

In [ ]:
# https://pypi.org/project/decorator/3.3.2/ 
# install decorator